In [1]:
import skfuzzy as fuzz
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from shapely.geometry import Polygon

In [3]:
df = pd.read_csv("test_Data.csv")

data = df[df.columns.values].values

data_ = np.array(data).transpose()


In [4]:
ncenters = 9
cluster = []

cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    data_, ncenters, 2
    , error = 0
    , maxiter = 1000
)

cluster_membership = np.argmax(u, 0)

for j in range(ncenters):
    cluster.append(data[cluster_membership == j])

print(cluster)

[array([[72.   , 88.   , 40.084],
       [89.   , 75.   , 40.76 ],
       [79.   , 76.   , 38.07 ],
       [55.   , 84.   , 37.358],
       [68.   , 80.   , 37.507],
       [73.   , 68.   , 35.739],
       [71.   , 80.   , 37.875],
       [77.   , 56.   , 35.502],
       [80.   , 78.   , 38.397]]), array([[ 79.   ,  42.   ,  35.848],
       [ 72.   ,  37.   ,  33.923],
       [ 78.   ,  56.   ,  35.769],
       [ 94.   ,  30.   ,  40.451],
       [ 93.   ,  44.   ,  40.187],
       [ 77.   ,  41.   ,  35.309],
       [ 93.   ,  39.   ,  40.168],
       [ 89.   ,  47.   ,  38.761],
       [ 92.   ,  32.   ,  39.66 ],
       [ 84.   ,  35.   ,  37.229],
       [ 91.   ,  30.   ,  39.199],
       [100.   ,  34.   ,  43.591]]), array([[53.   , 23.   , 26.104],
       [33.   , 20.   , 18.075],
       [55.   ,  9.   , 26.806],
       [49.   , 13.   , 22.101],
       [66.   ,  5.   , 32.039],
       [51.   , 17.   , 23.693],
       [34.   ,  7.   , 18.598],
       [60.   ,  2.   , 29.589],
  

In [5]:
def triangular_function(x, a, m, b):
    if x<a:
        return 0
    elif a<=x and x<=m:
        return (x-a)/(m-a)
    elif m<=x and x<=b:
        return (b-x)/(b-m)
    else:
        return 0

In [6]:
def arithmetic_mean(clusters, index_cluster, index_column):
    μ = 0
    n = len(clusters[index_cluster])
    for i in range(n):
        μ += clusters[index_cluster][i][index_column]
    μ = μ/n
    return μ

def standard_deviation(clusters, index_cluster, index_column, μ):
    σ = 0
    n = len(clusters[index_cluster])
    for i in range(n):
        σ += math.pow(clusters[index_cluster][i][index_column]-μ,2)
    σ = math.sqrt(σ/n)
    return σ

def tringular_fuzzy(clusters, index_cluster, index_column):
    μ = arithmetic_mean(clusters, index_cluster, index_column)
    σ = standard_deviation(clusters, index_cluster, index_column, μ)
    return [μ-σ,μ,μ+σ]

#   
def intersection(a, b):
    p1 = Polygon([(a[0],0),(a[1],1),(a[2],0)])
    p2 = Polygon([(b[0],0),(b[1],1),(b[2],0)])

    return p1.intersection(p2).area

def union(a, b):
    p1 = Polygon([(a[0],0),(a[1],1),(a[2],0)])
    p2 = Polygon([(b[0],0),(b[1],1),(b[2],0)])

    return p1.union(p2).area

def select_label_input1(triangle):
    labels = ['NOGREASY','MEDIUM','GREASY']
    ranges = [[0,0,50],[0,50,100],[50,100,100]]
    max_value = -1
    index = -1
    for i in range(len(ranges)):
        intersection_value = intersection(triangle,ranges[i])
        union_value = union(triangle,ranges[i])
        S = intersection_value/union_value
        if(S > max_value):
            max_value = S
            index = i
    #print(triangle,"=", ranges[index], labels[index])
    return labels[index]

def select_label_input2(triangle):
    labels = ['SMALL','MEDIUM','LONG']
    ranges = [[0,0,50],[0,50,100],[50,100,100]]
    max_value = -1
    index = -1
    for i in range(len(ranges)):
        intersection_value = intersection(triangle,ranges[i])
        union_value = union(triangle,ranges[i])
        S = intersection_value/union_value
        if(S > max_value):
            max_value = S
            index = i
    #print(triangle,"=", ranges[index], labels[index])
    return labels[index]

def select_label_output(triangle):
    labels = ['VERY_SHORT','SHORT','MEDIUM','LONG','VERY_LONG']
    ranges = [[0,8,12],[8,12,20],[12,20,35],[30,40,60],[45,60,60]]
    max_value = -1
    index = -1
    for i in range(len(ranges)):
        intersection_value = intersection(triangle,ranges[i])
        union_value = union(triangle,ranges[i])
        S = intersection_value/union_value
        if(S > max_value):
            max_value = S
            index = i
    #print(triangle,"=", ranges[index], labels[index])
    return labels[index]

for i in range(len(cluster)):
    input1 = tringular_fuzzy(cluster, i ,0)
    input2 = tringular_fuzzy(cluster, i ,1)
    output = tringular_fuzzy(cluster, i ,2)
    print('if Dirtness =',select_label_input2(input2),'and Typeofdirty =',select_label_input1(input1),'then Watch_time =',select_label_output(output))
    #print('if Typeofdirty =',select_label_input1(input1),'and Dirtness =',select_label_input2(input2),'then Watch_time =',select_label_output(output))
    #print('-------------------------------------------------------------------------------------------------------')

if Dirtness = LONG and Typeofdirty = GREASY then Watch_time = LONG
if Dirtness = MEDIUM and Typeofdirty = GREASY then Watch_time = LONG
if Dirtness = SMALL and Typeofdirty = MEDIUM then Watch_time = MEDIUM
if Dirtness = SMALL and Typeofdirty = NOGREASY then Watch_time = SHORT
if Dirtness = MEDIUM and Typeofdirty = MEDIUM then Watch_time = MEDIUM
if Dirtness = SMALL and Typeofdirty = GREASY then Watch_time = LONG
if Dirtness = LONG and Typeofdirty = GREASY then Watch_time = LONG
if Dirtness = LONG and Typeofdirty = NOGREASY then Watch_time = MEDIUM
if Dirtness = MEDIUM and Typeofdirty = NOGREASY then Watch_time = MEDIUM
